#  Homework - From REST to reasoning: ingest, index, and query with dlt and Cognee

📺 Video: https://www.youtube.com/watch?v=MNt_KK32gys

**Resources**

* [Slides](https://docs.google.com/presentation/d/1oHQilxEVqGGW4S2ctNEE0wHY2LgcjYLaRUziAoinsis/edit?usp=sharing)
* [Colab Notebook](https://colab.research.google.com/drive/1vBA9OIGChcKjjg8r5hHduR0v3A5D6rmH?usp=sharing) 


## Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

In [6]:
!pip install "dlt[qdrant]"

In [7]:
! pip install  "qdrant-client[fastembed]"

What's the version of dlt that you installed?

In [17]:
import dlt

dlt_version = dlt.__version__
print(f"dlt version: {dlt_version}")

dlt version: 1.14.1


## dlt Resourse

For reading the FAQ data, we have this helper function:

```python
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc
```

Annotate it with `@dlt.resource`. We will use it when creating
a dlt pipeline.

In [4]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

## Question 2. dlt pipeline

Now let's create a pipeline. 

We need to define a destination for that. Let's use the `qdrant` one:

In [3]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant"
)

In this case, we tell dlt (and Qdrant) to create a folder with
our data, and the name for it will be `db.qdrant`

Let's run it:


In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())

print(pipeline.last_trace)

Run started at 2025-07-22 17:41:16.527277+00:00 and COMPLETED in 3.95 seconds with 4 steps.
Step extract COMPLETED in 0.36 seconds.

Load package 1753206077.1719759 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)

Load package 1753206077.1719759 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 2.88 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.87 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Users/gabi/codes/llm_zoomcamp/03_workshop/db.qdrant location to store data
Load package 1753206077.1719759 is LOADED and contains no failed jobs

Step run COMPLETED in 3.95 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.87 seconds
1 load package(s) were loaded to destination qdrant and into data

How many rows were inserted into the `zoomcamp_data` collection?

Look for `"Normalized data for the following tables:"` in the trace output.

> 948

## Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the `meta.json` file created
in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2025/homework/dlt